In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.datasets import make_classification
RANDOM_STATE = 8
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score 
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from matplotlib import pyplot
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier


In [ ]:
#Importing and Splitting the dataset
dataset = pd.read_csv('gini_fs_final.csv')
df = dataset.iloc[ : , :]
#print(df.head())
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
print(X)
print(y)

     Patient age quantile  Hematocrit  Hemoglobin  Platelets  Red blood Cells  \
0                      17    0.236515   -0.022340  -0.517413         0.102004   
1                       1   -1.571682   -0.774212   1.429667        -0.850035   
2                       9   -0.747693   -0.586244  -0.429480        -1.361315   
3                      11    0.991838    0.792188   0.072992         0.542763   
4                       9    0.190738   -0.147652  -0.668155        -0.127191   
..                    ...         ...         ...        ...              ...   
595                    19    0.190738    0.165628  -0.102873         0.384090   
596                    19   -0.289922   -0.523588   0.663397         0.754327   
597                    15    0.717175    1.105468  -0.492289         0.613284   
598                    17   -3.242548   -2.779203  -1.773594        -3.318285   
599                    19    0.694287    0.541564  -0.906829         0.578024   

     Lymphocytes  Leukocyte

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
!pip install pyswarm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4481 sha256=4fba39719a8d1cbc21e24eb37f2b038fb87dc58c5f06267911a14de19314d079
  Stored in directory: /root/.cache/pip/wheels/ac/69/65/926e9c51b9fa99757cb43e8f6b74c5e6bb8b41a038b35c2db1
Successfully built pyswarm


In [ ]:
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor
from sklearn.svm import SVR, SVC


# Base class of models
class Model:
    def __init__(self, params, X_train, X_test, y_train, y_test, loss_func, model):
        self.params = params
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.loss_func = loss_func
        self.model = model

    # Method to get hyperparam type
    def get_types(self, param):
        pass

    # Given a key and a value, this methods returns a random value of a
    # specific hyperparam
    def get_params(self, key, value):
        pass

    # Generates initial model hyperparams
    def generate_params_model(self):
        p = []

        for i in self.params.keys():
            num = self.get_params(i, self.params[i])
            p.append(num)

        return p

    # Train a model and returns some loss function value
    def fit(self, params):
        params_dict = {}

        for i, key in enumerate(self.params.keys()):
            params_dict[key] = self.get_types(key)(params[i])

        model = self.model(**params_dict)
        model.fit(self.X_train, self.y_train)

        y_pred = model.predict(self.X_test)
        loss = self.loss_func(self.y_test, y_pred)

        return loss, model


class GradientBoosting(Model):
    def get_types(self, param):
        types = {
            'n_estimators': np.int64,
            'max_depth': np.int64,
            'learning_rate': np.float64,
            'tol': np.float64,
        }

        return types[param]

    def get_params(self, key, value):
        low = value[0]
        high = value[1]

        params = {
            'n_estimators': np.random.randint(low, high + 1),
            'max_depth': np.random.randint(low, high + 1),
            'learning_rate': np.random.uniform(low, high),
            'tol': np.random.uniform(low, high),
        }

        return params[key]


class Bagging(Model):
    def get_types(self, param):
        types = {
            'n_estimators': np.int64,
        }

        return types[param]

    def get_params(self, key, value):
        low = value[0]
        high = value[1]

        params = {
            'n_estimators': np.random.randint(low, high + 1),
            
        }

        return params[key]


class SupportVectorMachines(Model):
    def get_types(self, param):
        types = {
            'C': np.float64,
            'tol': np.float64
        }

        return types[param]

    def get_params(self, key, value):
        low = value[0]
        high = value[1]

        params = {
            'C': np.random.uniform(low, high),
            'tol': np.random.uniform(low, high),
        }

        return params[key]


def set_model(m, params, X_train, X_test, y_train, y_test, loss_func):
    if isinstance(m, GradientBoostingRegressor):
        model = GradientBoosting(params, X_train, X_test, y_train, y_test, loss_func, GradientBoostingRegressor)
    elif isinstance(m, GradientBoostingClassifier):
        model = GradientBoosting(params, X_train, X_test, y_train, y_test, loss_func, GradientBoostingClassifier)
    elif isinstance(m, SVR):
        model = SupportVectorMachines(params, X_train, X_test, y_train, y_test, loss_func, SVR)
    elif isinstance(m, SVC):
        model = SupportVectorMachines(params, X_train, X_test, y_train, y_test, loss_func, SVC)
    elif isinstance(m, BaggingClassifier ):
        model = Bagging(params, X_train, X_test, y_train, y_test, loss_func, BaggingClassifier)

    return model

In [ ]:
import time
import copy

import numpy as np

#from pso2.models import set_model


class Particle:
    def __init__(self, parameters, identifier):
        self.identifier = identifier            # Particle identifier
        self.position_i = parameters            # Particle position
        self.velocity_i = []                    # Particle velocity
        self.pos_best_i = []                    # Best position individual
        self.err_best_i = -1                    # Best error individual
        self.err_i = -1                         # Error individual
        self.model_i = None                     # Model individual
        self.num_dimensions = len(parameters)   # Num of problem dimensions

        # Generating initial velocities randomly
        for i in range(0, self.num_dimensions):
            v = np.random.rand(1)[0]
            self.velocity_i.append(v)

    def print_particle(self):
        print('particle_{}: {}'.format(self.identifier, self.position_i))

    def print_loss(self):
        print('loss_{}: {}'.format(self.identifier, self.err_i))

    # Evaluate current fitness
    def evaluate(self, model):
        self.err_i, self.model_i = model.fit(self.position_i)

        # Check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i == -1:
            self.pos_best_i = copy.deepcopy(self.position_i)
            self.err_best_i = self.err_i

    # Update new particle velocity
    def update_velocity(self, pos_best_g, c1, c2, w):
        """
        w: Constant inertia weight (how much to weigh the previous velocity)
        c1: Cognitive constant
        c2: Social constant
        """

        for i in range(0, self.num_dimensions):
            r1 = np.random.rand(1)[0]
            r2 = np.random.rand(1)[0]

            cognitive_vel = c1 * r1 * (self.pos_best_i[i] - self.position_i[i])
            social_vel = c2 * r2 * (pos_best_g[i] - self.position_i[i])

            self.velocity_i[i] = w * self.velocity_i[i] + cognitive_vel + social_vel

    # Update the particle position based off new velocity updates
    def update_position(self, boundaries):
        for i in range(0, self.num_dimensions):
            self.position_i[i] = self.position_i[i] + self.velocity_i[i]

            # Adjust maximum position if necessary
            if self.position_i[i] > boundaries[i][1]:
                self.position_i[i] = boundaries[i][1]

            # Adjsut minimum position if necessary
            if self.position_i[i] < boundaries[i][0]:
                self.position_i[i] = boundaries[i][0]


class PSO:
    def __init__(self, **params):
        self.c1 = params.get('c1')
        self.c2 = params.get('c2')
        self.w = params.get('w')
        self.max_iter = params.get('max_iter')
        self.n_pop = params.get('n_pop')
        self.boundaries = params.get('boundaries')
        self.loss_func = params.get('loss_func')
        self.verbose = params.get('verbose')

        self.err_best_g = -1        # Best error for group
        self.pos_best_g = []        # Best position for group
        self.model_best_g = None    # Best model for group
        self.best_params_ = {}

        # Machine Learning model
        self.model = set_model(
            m=params.get('model'),
            params=params.get('boundaries'),
            X_train=params.get('X_train'),
            X_test=params.get('X_test'),
            y_train=params.get('y_train'),
            y_test=params.get('y_test'),
            loss_func=params.get('loss_func')
        )

        # Establish the swarm
        self.swarm = []
        for i in range(0, self.n_pop):
            p = self.model.generate_params_model()
            particle = Particle(p, i)
            self.swarm.append(particle)

    def optimize(self):
        init = time.time()

        i = 0

        # Begin optimization loop
        while i < self.max_iter:
            if self.verbose:
                print('iter_{}'.format(i))

            # Cycle through particles in swarm and evaluate fitness
            for j in range(0, self.n_pop):
                self.swarm[j].evaluate(self.model)

                if self.verbose == 2:
                    self.swarm[j].print_loss()
                    self.swarm[j].print_particle()

                # Determine if current particle is the best (globally)
                if self.swarm[j].err_i < self.err_best_g or self.err_best_g == -1:
                    self.pos_best_g = copy.deepcopy(self.swarm[j].position_i)
                    self.err_best_g = self.swarm[j].err_i
                    self.model_best_g = copy.deepcopy(self.swarm[j].model_i)

            # Cycle through swarm and update velocities and positions
            for j in range(0, self.n_pop):
                self.swarm[j].update_velocity(self.pos_best_g, self.c1, self.c2, self.w)
                self.swarm[j].update_position(list(self.boundaries.values()))

            if self.verbose:
                print('best loss: {}'.format(self.err_best_g))

            i += 1

        end = time.time()

        if self.verbose:
            print('total time: {:.2f}s'.format(end - init))

        self.best_params_ = self.format_output()

        return self.model_best_g

    def format_output(self):
        output = {}

        for i, key in enumerate(self.boundaries.keys()):
            output[key] = self.model.get_types(key)(self.pos_best_g[i])

        return output

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = GradientBoostingRegressor()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.05973965940891693
iter_1
best loss: 0.059675854208935925
iter_2
best loss: 0.059675854208935925
iter_3
best loss: 0.059675854208935925
iter_4
best loss: 0.05940926330737765
iter_5
best loss: 0.05928679090767192
iter_6
best loss: 0.05928679090767192
iter_7
best loss: 0.0590419917616937
iter_8
best loss: 0.0590419917616937
iter_9
best loss: 0.0590419917616937
total time: 19.50s
{'n_estimators': 137, 'max_depth': 3, 'learning_rate': 0.1}


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = GradientBoostingClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.075
iter_1
best loss: 0.06666666666666667
iter_2
best loss: 0.058333333333333334
iter_3
best loss: 0.058333333333333334
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 16.63s
{'n_estimators': 69, 'max_depth': 3, 'learning_rate': 0.1}


In [ ]:
model = GradientBoostingClassifier(n_estimators = 69, max_depth = 3, learning_rate= 0.1,random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       106
           1       0.82      0.64      0.72        14

    accuracy                           0.94       120
   macro avg       0.89      0.81      0.84       120
weighted avg       0.94      0.94      0.94       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = RandomForestRegressor()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        #'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.059670546615611574
iter_1
best loss: 0.059663145535878113
iter_2
best loss: 0.059663145535878113
iter_3
best loss: 0.058913098058070726
iter_4
best loss: 0.058913098058070726
iter_5
best loss: 0.05888149946562917
iter_6
best loss: 0.05888149946562917
iter_7
best loss: 0.05888149946562917
iter_8
best loss: 0.05888149946562917
iter_9
best loss: 0.05888149946562917
total time: 14.09s
{'n_estimators': 60, 'max_depth': 3}


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = RandomForestClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        #'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.08333333333333333
iter_1
best loss: 0.058333333333333334
iter_2
best loss: 0.058333333333333334
iter_3
best loss: 0.058333333333333334
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 28.46s
{'n_estimators': 150, 'max_depth': 3}


In [ ]:
model = RandomForestClassifier(n_estimators = 150, max_depth = 3,random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       106
           1       1.00      0.21      0.35        14

    accuracy                           0.91       120
   macro avg       0.95      0.61      0.65       120
weighted avg       0.92      0.91      0.88       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = AdaBoostClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        #'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.058333333333333334
iter_1
best loss: 0.058333333333333334
iter_2
best loss: 0.058333333333333334
iter_3
best loss: 0.058333333333333334
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 27.44s
{'n_estimators': 150, 'max_depth': 3}


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = AdaBoostClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.06666666666666667
iter_1
best loss: 0.06666666666666667
iter_2
best loss: 0.058333333333333334
iter_3
best loss: 0.058333333333333334
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 20.48s
{'n_estimators': 82, 'max_depth': 3, 'learning_rate': 0.05384606852207195}


In [ ]:
model = AdaBoostClassifier(n_estimators = 82, learning_rate = 0.05384606852207195,random_state=8)
#96 0.06897974961663625
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       106
           1       0.71      0.36      0.48        14

    accuracy                           0.91       120
   macro avg       0.82      0.67      0.71       120
weighted avg       0.90      0.91      0.89       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = XGBClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.08333333333333333
iter_1
best loss: 0.06666666666666667
iter_2
best loss: 0.06666666666666667
iter_3
best loss: 0.058333333333333334
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 19.96s
{'n_estimators': 96, 'max_depth': 3, 'learning_rate': 0.06897974961663625}


In [ ]:
model = XGBClassifier(n_estimators = 96, learning_rate = 0.06897974961663625,max_depth=3,random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       106
           1       0.64      0.64      0.64        14

    accuracy                           0.92       120
   macro avg       0.80      0.80      0.80       120
weighted avg       0.92      0.92      0.92       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = XGBClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.058333333333333334
iter_1
best loss: 0.058333333333333334
iter_2
best loss: 0.058333333333333334
iter_3
best loss: 0.05
iter_4
best loss: 0.05
iter_5
best loss: 0.05
iter_6
best loss: 0.05
iter_7
best loss: 0.05
iter_8
best loss: 0.05
iter_9
best loss: 0.05
total time: 5.24s
{'n_estimators': 15, 'max_depth': 13, 'learning_rate': 0.09721381442041677}


In [ ]:
model = XGBClassifier(n_estimators = 15, learning_rate = 0.09721381442041677,max_depth=13,random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       106
           1       0.83      0.71      0.77        14

    accuracy                           0.95       120
   macro avg       0.90      0.85      0.87       120
weighted avg       0.95      0.95      0.95       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = AdaBoostClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.08333333333333333
iter_1
best loss: 0.08333333333333333
iter_2
best loss: 0.08333333333333333
iter_3
best loss: 0.08333333333333333
iter_4
best loss: 0.08333333333333333
iter_5
best loss: 0.08333333333333333
iter_6
best loss: 0.08333333333333333
iter_7
best loss: 0.08333333333333333
iter_8
best loss: 0.08333333333333333
iter_9
best loss: 0.08333333333333333
total time: 15.19s
{'n_estimators': 74, 'learning_rate': 0.08921146119789608}


In [ ]:
model = AdaBoostClassifier(n_estimators = 74, learning_rate = 0.08921146119789608,random_state=8)
#40 0.1 92
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.93      0.98      0.95       106
           1       0.75      0.43      0.55        14

    accuracy                           0.92       120
   macro avg       0.84      0.70      0.75       120
weighted avg       0.91      0.92      0.91       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = RandomForestClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
        'max_depth': (3, 15),
        #'learning_rate': (0.001, 0.1)
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.06666666666666667
iter_1
best loss: 0.06666666666666667
iter_2
best loss: 0.06666666666666667
iter_3
best loss: 0.06666666666666667
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 12.18s
{'n_estimators': 59, 'max_depth': 8}


In [ ]:
model = RandomForestClassifier(n_estimators = 59, max_depth = 8,random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.94      0.99      0.96       106
           1       0.88      0.50      0.64        14

    accuracy                           0.93       120
   macro avg       0.91      0.75      0.80       120
weighted avg       0.93      0.93      0.93       120
 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

#from pso2.optimizer import PSO



if __name__ == '__main__':
    # Load and dataprep
    

    # Model
    model = BaggingClassifier()

    # Boundaries
    boundaries = {
        'n_estimators': (10, 200),
    }

    # Function to minimize (or maximize)
    loss_func = mean_squared_error

    # PSO params
    c1 = 1.0
    c2 = 2.0
    w = 0.5
    n_pop = 10
    max_iter = 10

    # Compress all parameters into a dict
    pso_params = {
        'model': model,
        'boundaries': boundaries,
        'loss_func': loss_func,
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'c1': c1,
        'c2': c2,
        'w': w,
        'n_pop': n_pop,
        'max_iter': max_iter,
        'verbose': 1,
    }

    opt = PSO(**pso_params)
    opt.optimize()
    print(opt.best_params_)

iter_0
best loss: 0.06666666666666667
iter_1
best loss: 0.058333333333333334
iter_2
best loss: 0.058333333333333334
iter_3
best loss: 0.058333333333333334
iter_4
best loss: 0.058333333333333334
iter_5
best loss: 0.058333333333333334
iter_6
best loss: 0.058333333333333334
iter_7
best loss: 0.058333333333333334
iter_8
best loss: 0.058333333333333334
iter_9
best loss: 0.058333333333333334
total time: 27.35s
{'n_estimators': 75}


In [ ]:
model = BaggingClassifier(n_estimators = 75,random_state=8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test , y_pred),"\n")

              precision    recall  f1-score   support

           0       0.94      0.97      0.96       106
           1       0.73      0.57      0.64        14

    accuracy                           0.93       120
   macro avg       0.84      0.77      0.80       120
weighted avg       0.92      0.93      0.92       120
 

